# Course Notes
Use this workspace to take notes, store code snippets, and build your own interactive cheatsheet!

_Note that the data from the course is not yet added to this workspace. You will need to navigate to the course overview page, download any data you wish to use, and add it to the file browser._

In [6]:
# Import any packages you want to use here

import pandas as pd
import numpy as np
taxi_veh = pd.read_csv('taxi_vehicles.csv')
taxi_owners = pd.read_csv('taxi_owners.csv')
biz_owners = pd.read_csv('biz_owners.csv')
licenses = pd.read_csv('licenses.csv')
ridership = pd.read_csv('ridership.csv')
stations = pd.read_csv('stations.csv')
cal = pd.read_csv('cal.csv')

#soccer_names = np.genfromtxt(
#   fname="soccer.csv",
#   delimiter=",",
#    usecols=1,
#   skip_header=1,
#   dtype=str,
#   encoding="utf", 
# )


## Take Notes

Add notes here about the concepts you've learned and code cells with code you want to keep.

## Inner Join

pandas has a DataFrame method with joins table called .merge()

df1.merge(df2, on='column_a') will joing df1 on df2, matching based on the column column_a. When you print the variable this joined table has been assigned to, it will list the content of df1 first on the left, then df2. This join will show only the rows where the contents of column_a match in df1 and df2: an inner join. If there are columns with the same name between the 2 DataFrames, the merge will automatically add a suffix. Use the suffixes= argument to pass a tuple of suffixes that you want to use for the columns. 

In [3]:
# Merge the taxi_owners and taxi_veh tables setting a suffix
taxi_own_veh = taxi_owners.merge(taxi_veh, on='vid', suffixes=('_own', '_veh'))

# Print the column names of the taxi_own_veh
print(taxi_own_veh.columns)

# Print the value_counts to find the most popular fuel_type
print(taxi_own_veh['fuel_type'].value_counts())

Index(['Unnamed: 0_own', 'rid', 'vid', 'owner_own', 'address', 'zip',
       'Unnamed: 0_veh', 'make', 'model', 'year', 'fuel_type', 'owner_veh'],
      dtype='object')
HYBRID                    2792
GASOLINE                   611
FLEX FUEL                   89
COMPRESSED NATURAL GAS      27
Name: fuel_type, dtype: int64


In [ ]:
# Merge the wards and census tables on the ward column
wards_census = wards.merge(census, on='ward')

# Print the shape of wards_census
print('wards_census table shape:', wards_census.shape)

# Aggregate Functions

In the .agg funtion below it includes a dictionary in the argument
Here's notes on why that works:
Here we want to create a new dataframe that sums the sales and counts the order numbers for each category in an existing dataframe.

We create a dictionary containing the columns we want to aggregate as keys and the aggregate we want to perform as the value. This can then be passed into the groupby agg function to create the new aggregated dataframe. (https://www.datasnips.com/148/how-to-create-different-aggregations-using-groupby-agg-dictionaries/)

In [5]:
# One-to-many merge
# Merge the licenses and biz_owners table on account
licenses_owners = licenses.merge(biz_owners, on='account')

# Group the results by title then count the number of accounts
counted_df = licenses_owners.groupby('title').agg({'account':'count'})

# Sort the counted_df in desending order
sorted_df = counted_df.sort_values('account', ascending=False)

# Use .head() method to print the first few rows of sorted_df
print(sorted_df.head())


                 account
title                   
PRESIDENT           6259
SECRETARY           5205
SOLE PROPRIETOR     1658
OTHER               1200
VICE PRESIDENT       970


## Merging multiple DataFrames

A list of column names can be passed into the on= argument to further specify the merge

In [ ]:
df1.merge(df2, on='col') \ # \ symbol lets code continue on next line
    .merge(df3, on='col') \ # merge onto other DataFrames
    .merge(df4, on='col')

In [ ]:
# Merge the ridership, cal, and stations tables
ridership_cal_stations = ridership.merge(cal, on=['year','month','day']) \
							.merge(stations, on='station_id')

# Create a filter to filter ridership_cal_stations
filter_criteria = ((ridership_cal_stations['month'] == 7) 
                   & (ridership_cal_stations['day_type'] == 'Weekday') 
                   & (ridership_cal_stations['station_name'] == 'Wilson'))

# Use .loc and the filter to select for rides
print(ridership_cal_stations.loc[filter_criteria, 'rides'].sum())

In [7]:
# Merge the ridership, cal, and stations tables
ridership_cal_stations = ridership.merge(cal, on=['year','month','day']) \
							.merge(stations, on='station_id')

# Create a filter to filter ridership_cal_stations
filter_criteria = ((ridership_cal_stations['month'] == 7) 
                   & (ridership_cal_stations['day_type'] == 'Weekday') 
                   & (ridership_cal_stations['station_name'] == 'Wilson'))

# Use .loc and the filter to select for rides
print(ridership_cal_stations.loc[filter_criteria, 'rides'].sum())

140005


# **_Grouping by values and counting_**


In [ ]:
# Count the number of genres
genre_count = genres_movies.groupby('genre').agg({'id':'count'})

## Merging a table to itself (self join)

There might be a column that refers to another column in the table. For instance, in a table of movies, there might be a 'sequel' column that contains an id number that refers to the 'id' field of the sequel movie. So imagine there's a left and right table that are duplicates of the one table. This will be an inner join. 

The merge will have this format:
sequels.merge(sequels, left_on='sequel', right_on='id', suffixes='_org', '_seq')_ So the left table is treated like the original. This shows only movies with sequels because it's an inner join

We can use the different types of joins with a self join:
sequels.merge(sequels, left_on='sequel', right_on='id', how='left', suffixes='_org', '_seq')_ So it only shows all movies. 

You might need self joins when working with hierarchical relationships (like employees and managers), or sequential movements like logistic movements, graph data like networks of friends

In [ ]:
# Merge the crews table to itself
crews_self_merged = crews.merge(crews, on='id', how='inner',
                                suffixes=('_dir','_crew'))

# Create a boolean index to select the appropriate rows
# This selects rows with 'Director' in the left table, but not in the right
# table, so it removes job comibinations that aren't with the director
boolean_filter = ((crews_self_merged['job_dir'] == 'Director') & 
                  (crews_self_merged['job_crew'] != 'Director'))
direct_crews = crews_self_merged[boolean_filter]

# Print the first few rows of direct_crews
print(direct_crews.head())

## Merging on indexes

If you're reading data from a csv file, you can use the index_col= argument of the read_csv method to set the index to a column, like ['id']. You can have multiindex datasets, which can also be listed in the on= argument of merge. When you're merging on indexes, you have to set the left_index and right_index arguments to True

In [ ]:
# Merge to the movies table the ratings table on the index
movies_ratings = movies.merge(ratings, how='left', on='id')

# Print the first few rows of movies_ratings
print(movies_ratings.head())

In [ ]:
# Merge sequels and financials on index id
sequels_fin = sequels.merge(financials, on='id', how='left')

# Self merge with suffixes as inner join with left on sequel and right on id
orig_seq = sequels_fin.merge(sequels_fin, how='inner', left_on='sequel', 
                             right_on='id', right_index=True,
                             suffixes=('_org','_seq'))

# Add calculation to subtract revenue_org from revenue_seq 
orig_seq['diff'] = orig_seq['revenue_seq'] - orig_seq['revenue_org']

# Select the title_org, title_seq, and diff 
titles_diff = orig_seq[['title_org','title_seq','diff']]

# Print the first rows of the sorted titles_diff
print(titles_diff.sort_values(by='diff', ascending=False).head())

## Filtering Join

Everything so far has been a mutating join, which combines data from two tables based on matching observations in both tables. A filtering join filter observations from _one_ table based on whether or not they match an observation in _another_ table. 

A semi join is one. It filters the left table down to those observations that have a match in the right table. Like an inner join, only the interection between the tables is returned, but only columns from the left table are shown, and no columns from the right table. No duplicate records are returned from the left either, even if there is a one to many relationship. First merge 2 table with an inner join. 

In [ ]:
genres_tracks = genres.merge(top_tracks, on='gid')

# .isin() will compare every 'gid' in the genres table to the 'gid' in genres_tracks table
genres['gid'].isin(genres_tracks['gid'])

# Will return Boolean series of True and False values; have to subset the dataset with it
top_genres = genres[genres['gid'].isin(genres_tracks['gid'])]

An anti join returns the observations in the left table that do not have a matching observation in the right table. Only returns columns from left table, not the right table. First do a left join returning all values from the left table

In [ ]:
genre_tracks = genres.merge(top_tracks, on='gid', how='left', indicator=True)

# Setting indicator to True generates a _merge column that show the source of each row, like 'both' or 'left_only'

# Using the loc accessor and the '_merge' column lets you select the rows that only come from the left table, and then only returns the 'gid' column
gid_list = genres_tracks.loc[genres_tracks['_merge'] == 'left_only', 'gid']

# the .isin() method will filter the rows with gids in the gid_list
non_top_genres = genres[genres['gid'].isin(gid_list)]

# Create an index that returns true if name_1 or name_2 are null


In [ ]:
# Create an index that returns true if name_1 or name_2 are null
m = ((iron_1_and_2['name_1'].isnull()) | 
     (iron_1_and_2['name_2'].isnull()))

# Subsetting Data by Columns

In [ ]:
# Subset the gdp and returns columns
gdp_returns = gdp_sp500[['gdp', 'returns']]

# Creating a Status Flag 

In [ ]:
# Create a list based on the row value of gdp_recession['econ_status'], using a list comprehension and a conditional expression
is_recession = ['r' if s=='recession' else 'g' for s in gdp_recession['econ_status']]

# Adding New Column

In [ ]:
# To add a column to an existing table there are a number of methods. One approach is to use a format similar to tb1['new_column_name'] = 'some operation'.
gdp_pop['gdp_per_capita'] = gdp_pop['gdp'] / gdp_pop['pop']